# Analytics

Here I am going to pry into some baseball reference data that is available online. I'm going to try to answer the question of whether there are 3 players that, taken together, have played for all 30 teams.

In [2]:
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from bs4 import BeautifulSoup as bs
from lxml import html
import requests
from urllib2 import urlopen
import numpy as np

First things first, we download the page of players with the most franchises played for.

In [3]:
url = "http://www.baseball-reference.com/leaders/leaders_most_franchises.shtml"
soup = bs(urlopen(url),"html.parser")

In [4]:
players = soup.find("table",attrs={"class":"stats_table sortable"})
headings = [th.get_text() for th in players.find("tr").find_all("th")]

In [5]:
pdata = []
pdata.append(headings)
for row in players.find_all("tr")[1:]:
    pdata_row = (td for td in row.find_all("td"))
    pdata.append(pdata_row)
index = np.arange(len(pdata)-1)

In [6]:
playerdf_raw = pd.DataFrame(data=pdata[1:][0:],index=index,columns=pdata[0][0:])
playerdf_raw.head()

,Player,Years Played,Franchises,Leagues
0,"<td><a href=""/players/d/doteloc01.shtml"">Octav...","<td align=""RIGHT"">1999-2013</td>","<td align=""RIGHT"">13</td>","<td align=""RIGHT"">2</td>"
1,"<td><a href=""/players/m/morgami01.shtml"">Mike ...","<td align=""RIGHT"">1978-2002</td>","<td align=""RIGHT"">12</td>","<td align=""RIGHT"">2</td>"
2,"<td><a href=""/players/s/stairma01.shtml"">Matt ...","<td align=""RIGHT"">1992-2011</td>","<td align=""RIGHT"">12</td>","<td align=""RIGHT"">2</td>"
3,"<td><a href=""/players/v/villoro01.shtml"">Ron V...","<td align=""RIGHT"">1995-2009</td>","<td align=""RIGHT"">12</td>","<td align=""RIGHT"">2</td>"
4,"<td><a href=""/players/w/weyhigu01.shtml"">Gus W...","<td align=""RIGHT"">1887-1901</td>","<td align=""RIGHT"">11</td>","<td align=""RIGHT"">4</td>"


In [7]:
print playerdf_raw['Player'][0]

<td><a href="/players/d/doteloc01.shtml">Octavio Dotel</a></td>


It's already ordered by franchises, and the webpages of each player is embedded within the player's names, which kind of sucks. So let's build a new table with everything correctly formatted.

In [8]:
pdata_better = []
pdata_better.append(["Name","Page","Franchises"])
base = "www.baseball-reference.com"
for i in index:
    pname = playerdf_raw['Player'][i].get_text()
    ppage = base+str(playerdf_raw['Player'][i]).split("\"")[1]
    pfnum = int(playerdf_raw['Franchises'][i].get_text())
    row = (pname,ppage,pfnum)
    pdata_better.append(row)
index = np.arange(len(pdata_better)-1)
playerdf = pd.DataFrame(data=pdata_better[1:][0:],index=index,columns=pdata_better[0][0:])
playerdf.head()

,Name,Page,Franchises
0,Octavio Dotel,www.baseball-reference.com/players/d/doteloc01...,13
1,Mike Morgan,www.baseball-reference.com/players/m/morgami01...,12
2,Matt Stairs,www.baseball-reference.com/players/s/stairma01...,12
3,Ron Villone,www.baseball-reference.com/players/v/villoro01...,12
4,Gus Weyhing,www.baseball-reference.com/players/w/weyhigu01...,11


Great! Now we have all the players, their pages, and their number of franchises. Next we have to write a function that takes in a webpage, pulls out the the team list and spits it back.

In [9]:
def get_teams(page):
    playersoup = bs(urlopen("http://"+page),"html.parser")
    pvbtable = playersoup.find("table",attrs={"class":"sortable stats_table row_summable","id":"batting_value"})
    teams = set()
    for row in pvbtable.find_all("tr")[1:]:
        if len(row.find_all("td")) < 24: continue
        year = row.find("td").get_text()
        #Sometimes rows are blank or there is extra text there
        if len(year)==4: year = int(year)
        elif len(year) < 4: continue
        else: year = int(year[:4])
        links = row.find("a")
        if len(links)>0 and year>1901: #Arbitrary cutoff
            teams.add(links.get_text())
    return teams
output = get_teams(playerdf['Page'][0])
print output

set([u'NYY', u'STL', u'LAD', u'DET', u'ATL', u'NYM', u'OAK', u'TOR', u'HOU', u'CHW', u'KCR', u'PIT', u'COL'])


Woohoo! Now we can get the teams for any player that we want. I will now design a function that takes in three sets, figures out unique teams among them, and then outputs some useful information. But first I'll write a function that checks a set for duplicate franchises and cleans it out.

In [10]:
#Let's get some test data. Here are the top 3 most franchised players.
teams1 = get_teams(playerdf['Page'][0]) #Dotel
teams2 = get_teams(playerdf['Page'][23]) #Branyan
teams3 = get_teams(playerdf['Page'][2]) #The third guy

In [11]:
#The latter abbreviation should always be the old one to be removed
def clean_duplicates(teams):
    if 'MON' in teams and 'WSN' in teams: teams.remove('MON')
    if 'MIN' in teams and 'WHS' in teams: teams.remove('WHS')
    if 'MIA' in teams and 'FLA' in teams: teams.remove('FLA')
    if 'TBR' in teams and 'TBR' in teams: teams.remove('TBR')
    if 'LAA' in teams and 'CAL' in teams: teams.remove('CAL')
    if 'ANA' in teams and 'CAL' in teams: 
        teams.remove('ANA')
        teams.remove('CAL')
        teams.add('LAA')
    if 'LAA' in teams and 'ANA' in teams: teams.remove('ANA')
    if 'MIL' in teams and 'SEP' in teams: teams.remove('SEP')
    if 'KCA' in teams and 'PHA' in teams: 
        teams.remove('PHA')
        teams.remove('KCA')
        teams.add('OAK')
    if 'OAK' in teams and 'KCA' in teams: teams.remove('KCA')
    if 'OAK' in teams and 'PHA' in teams: teams.remove('PHA')
    if 'ATL' in teams and 'MLN' in teams: teams.remove('MLN')
    if 'MLN' in teams and 'BSN' in teams: 
        teams.remove('BSN')
        teams.remove('MLN')
        teams.add('ATL')
    if 'ATL' in teams and 'BSN' in teams: teams.remove('BSN')
    if 'LAD' in teams and 'BRO' in teams: teams.remove('BRO')
    #Houston had the same initials even when they were the Colt .45s
    return teams

def clean_all_teams(teams_all): #Used to get rid of duplicates en masse
    for team in teams_all:
        clean_duplicates(team)
    return

Let's make a function to combine team lists and then clean them of duplicates.

In [12]:
def get_intersection(teams1,teams2,teams3):
    teams_all = set().union(teams1,teams2,teams3)
    teams_all = clean_duplicates(teams_all)
    return teams_all
teams_all = get_intersection(teams1,teams2,set()) # Quick test
print len(teams_all)

22


# READ THIS BEFORE YOU PROCEED!!!

It's time to assemble the master list of people's teams. 

If someone is running this to experiment, you should uncomment the if statement and only do a first small batch so that it doesn't take forever to run. It takes forever to run if you do the whole list.

In [13]:
player_teams = []
for i,Page in zip(range(len(playerdf['Page'])),playerdf['Page']):
    if i > 30: break #Comment this line out by adding a # to do it all
    player_teams.append(get_teams(Page))
print player_teams[:3]

[set([u'NYY', u'STL', u'LAD', u'DET', u'ATL', u'NYM', u'OAK', u'TOR', u'HOU', u'CHW', u'KCR', u'PIT', u'COL']), set([u'NYY', u'STL', u'MIN', u'TOR', u'OAK', u'CIN', u'TEX', u'LAD', u'CHC', u'SEA', u'ARI', u'BAL']), set([u'PHI', u'CHC', u'SDP', u'MIL', u'TOR', u'DET', u'BOS', u'OAK', u'WSN', u'TEX', u'MON', u'KCR', u'PIT'])]


Do a triple loop over the player_teams list and figure out the largest set.

In [15]:
clean_all_teams(player_teams)

maxteams = 0 #This holds how many teams have been played on
biggest_set = None #This holds the list of teams of the first biggest group found
set_indices = None #This holds indices of the best set
all_sets_indices = [] #This holds all the indices of the best sets
all_sets_names = [] #This holds the names of all the best sets
set_names = []
for i in xrange(0,len(player_teams)):
    for j in xrange(i+1,len(player_teams)):
        for k in xrange(j+1,len(player_teams)):
            combo = get_intersection(player_teams[i],player_teams[j],player_teams[k])
            if len(combo) == maxteams:
                all_sets_indices.append((i,j,k))
                all_sets_names.append([playerdf['Name'][i],playerdf['Name'][j],playerdf['Name'][k]])

            if len(combo) > maxteams:
                maxteams = len(combo)
                biggest_set = combo
                set_indices = (i,j,k)
                set_names =[playerdf['Name'][i],playerdf['Name'][j],playerdf['Name'][k]]
                all_sets_indices = [set_indices]
                all_sets_names = [set_names]

print biggest_set
print maxteams, set_indices
print set_names[0]+" | "+set_names[1]+" | "+set_names[2]
print best_set_indices
for bsn in best_set_names:
    print bsn[0]," | ", bsn[1]," | ",bsn[2]

set([u'BOS', u'NYY', u'MIL', u'TOR', u'DET', u'ATL', u'NYM', u'OAK', u'CIN', u'WSN', u'HOU', u'CHW', u'FLA', u'COL', u'PHI', u'ARI', u'SDP', u'LAD', u'LAA', u'CLE', u'STL', u'CHC', u'SEA', u'KCR', u'SFG', u'PIT', u'TBD'])
27 (0, 15, 23)
Octavio Dotel | Julian Tavarez | Russell Branyan
[(0, 15, 23), (0, 17, 23), (1, 3, 30), (2, 13, 16)]
Octavio Dotel  |  Julian Tavarez  |  Russell Branyan
Octavio Dotel  |  Todd Zeile  |  Russell Branyan
Mike Morgan  |  Ron Villone  |  Roberto Hernandez
Matt Stairs  |  Terry Mulholland  |  Rick White
